In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
logger = logging.getLogger("noisepy.seis.io")
logger.setLevel("INFO")

In [ ]:
from noisepy.seis.io.h5store import DASH5DataStore

In [ ]:
DAS_DATA = "../../tests/das/"

raw_store = DASH5DataStore(path = DAS_DATA,
                           sampling_rate=100,
                           channel_numbers=[0,1,2],
                           file_naming = "%Y-%m-%d-%H-%M-%S.h5",
                           date_range = None)
raw_store.fs

In [ ]:
span = raw_store.get_timespans()
channels = raw_store.get_channels(span[0])
d = raw_store.read_data(span[0], channels[0])
d.stream